In [1]:
#!pip3 install opencv-python
from torchvision import models
import torch.nn as nn
from torch.utils.data import DataLoader
from utils import data_loader as dl

# Load your own dataset
data_loader = dl.DataLoader()
train_data, test_data = data_loader.get_dataset("car")

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64)

# Step 1:  Load a pre-trained model
model = models.resnet50(weights='IMAGENET1K_V1')

# Step 2: Replace the Classifier Layers
num_classes = 2
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)

# Step 3: Free the pre-trained (feature extraction) Layers

for param in model.parameters():
    param.requires_grad = False

# Only the classifier layer' weights updated
for param in model.fc.parameters():
    param.requires_grad = True

Loading dataset: car...


In [2]:
import time
import torch
import torch.optim as optim

print("CUDA available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A")

CUDA available: True
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [3]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from pytorchcv.model_provider import get_model as ptcv_get_model  # Third-party provider

# Check CUDA availability
print("CUDA available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "N/A")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load MobileNetV1
model = ptcv_get_model("mobilenet_w1", pretrained=True)  # Load pretrained MobileNetV1
model.to(device)

# Set up loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Assuming train_loader is already defined elsewhere in your script
num_epochs = 5 #10
start_time = time.time()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

print("Finished fine-tuning")
computation_time = time.time() - start_time
print(f"Computation Time: {computation_time} seconds")


CUDA available: True
GPU Name: NVIDIA GeForce RTX 3050 Laptop GPU
Using device: cuda


KeyboardInterrupt: 

In [4]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.eval()

correct = 0
total = 0
y_pred = []
y_true = []
with torch.no_grad():
    for inputs, labels in test_loader:
        # Move inputs and labels to the same device as the model
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        prob, predicted = torch.max(outputs, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        y_pred.extend(predicted.cpu())  # Move predicted values back to CPU for further processing
        y_true.extend(labels.cpu())    # Move true labels back to CPU for further processing

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:0.3f}%')


Accuracy: 96.756%


In [4]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np

y_pred_list = [item.item() for item in y_pred]  # Using .item() to get the scalar value from each tensor
y_true_list = [item.item() for item in y_true]

class_labels = pd.DataFrame({'Predicted': y_pred_list, 'Truth_Label': y_true_list})
class_counts = class_labels['Truth_Label'].value_counts().sort_index()
cm = confusion_matrix(y_true_list, y_pred_list)

print("Confusion Matrix:")
print(cm)

TP = np.diag(cm)  
FP = np.sum(cm, axis=0) - TP
FN = np.sum(cm, axis=1) - TP

result_df = pd.DataFrame({"True Positive": TP, "False Positive": FP, "False Negative": FN, "Number_Samples": class_counts.values})
print(f"Accuracy of the model is: {100*result_df['True Positive'].sum()/len(class_labels):0.2f}%")

NameError: name 'y_pred' is not defined

In [6]:
#export mobilenetv1 to onnx

import torch

# Make sure the model is on CPU for export
model = model.to('cpu')
model.eval()

# Dummy input (adjust shape as needed; MobileNet expects 3x224x224)
dummy_input = torch.randn(1, 3, 224, 224)

torch.onnx.export(model, dummy_input, "mobilenetv1-02.onnx", 
                   verbose=True, 
                   input_names=["input"],
                   output_names=["output"],
                   opset_version=11)  # opset_version can be 11 or newer
